In [3]:
import pandas as pd
import numpy as np
import os

def load_and_clean_data(pr_file_path, repo_file_path, output_path):
    print("Step 1: Loading datasets...")
    
    # 1. Load two Parquet files
    try:
        # Core Data: PR records (contains Agent, State, Body, Time)
        print(f"Reading {pr_file_path}...")
        df_pr = pd.read_parquet(pr_file_path)
        print(f"Loaded PR Data: {df_pr.shape} rows")
        
        # Auxiliary Data: Repository info (contains Language, Stars)
        print(f"Reading {repo_file_path}...")
        df_repo = pd.read_parquet(repo_file_path)
        print(f"Loaded Repo Data: {df_repo.shape} rows")
        
    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Please ensure the .parquet files are in the same directory as this script.")
        return

    # ==========================================
    # 2. Merge Datasets
    # ==========================================
    print("Step 2: Merging datasets...")
    
    # Automatically find columns to merge on (usually repo_name or repo_id)
    common_cols = list(set(df_pr.columns) & set(df_repo.columns))
    print(f"Common columns found: {common_cols}")
    
    if not common_cols:
        print("Error: No common columns to merge on!")
        return
        
    # Merge on the first common column found
    merge_key = 'repo_name' if 'repo_name' in common_cols else common_cols[0]
    print(f"Merging on '{merge_key}'...")
    
    # Merge! (Left Join: Keep all PR records, attach corresponding Repo info)
    df = pd.merge(df_pr, df_repo, on=merge_key, how='left')
    print(f"Merged shape: {df.shape}")

    # ==========================================
    # 3. Data Cleaning & Feature Engineering
    # ==========================================
    print("Step 3: Cleaning & Feature Engineering...")

    # A. Filtering
    # We must have Agent Name and PR State for the analysis.
    target_cols = ['agent_name', 'state', 'created_at', 'closed_at']
    
    existing_cols = [c for c in target_cols if c in df.columns]
    df = df.dropna(subset=existing_cols)

    # B. Feature Engineering: Time-to-Decision
    if 'created_at' in df.columns and 'closed_at' in df.columns:
        # Convert to datetime objects
        df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
        df['closed_at'] = pd.to_datetime(df['closed_at'], errors='coerce')
        
        # Calculate duration in hours
        df['decision_time_hours'] = (df['closed_at'] - df['created_at']).dt.total_seconds() / 3600
        
        # Filter out invalid times
        df = df[df['decision_time_hours'] >= 0]

    # C. Feature Engineering: Word Count
    if 'body' in df.columns:
        df['body_word_count'] = df['body'].astype(str).fillna("").apply(lambda x: len(x.split()))

    # D. Feature Engineering: Binary State Encoding
    if 'state' in df.columns:
        df['is_merged'] = df['state'].astype(str).str.lower().apply(lambda x: 1 if 'merged' in x else 0)

    # E. Feature Engineering: Language Binning
    if 'language' in df.columns:
        top_langs = df['language'].value_counts().nlargest(10).index
        df['language_grouped'] = df['language'].apply(lambda x: x if x in top_langs else 'Other')

    # ==========================================
    # 4. Save Results
    # ==========================================
    df.to_csv(output_path, index=False)
    print(f"Success! Cleaned data saved to: {output_path}")

if __name__ == "__main__":
    # Define file paths (assuming files are in the CURRENT directory)
    pr_file = 'all_pull_request.parquet'
    repo_file = 'all_repository.parquet'
    output_file = 'cleaned_data.csv'
    
    load_and_clean_data(pr_file, repo_file, output_file)

Step 1: Loading datasets...
Reading all_pull_request.parquet...
Loaded PR Data: (932791, 14) rows
Reading all_repository.parquet...
Loaded Repo Data: (116211, 7) rows
Step 2: Merging datasets...
Common columns found: ['id']
Merging on 'id'...
Merged shape: (932791, 20)
Step 3: Cleaning & Feature Engineering...
Success! Cleaned data saved to: cleaned_data.csv
